In [39]:
!nvidia-smi

Sun Aug 14 15:39:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P8    19W / 210W |    972MiB /  8192MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 資料讀取與前處理

In [6]:
###### 資料讀取 實際測試時此區改為讀取攔截之封包資料
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

csv_files = []
for dirname, _, filenames in os.walk('MachineLearningCSV/MachineLearningCVE'):
    for filename in filenames:
        csv_file = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))
        csv_files.append(csv_file)

# df = pd.concat([pd.read_csv(file) for file in csv_files[:8]], ignore_index=True)
df = pd.read_csv("MachineLearningCSV/MachineLearningCVE/week_2.csv",index_col = 0)

df.columns = df.columns.str.strip()
print("original length of df:", len(df))
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
print("after droping null values, the length of df:", len(df))

MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\week.csv
MachineLearningCSV/MachineLearningCVE\week_2.csv
original length of df: 2830743
after droping null values, the length of df: 2827876


In [7]:
###### 針對標籤之處理，實際測試攔截封包資料時可拿掉
df_experiment = df.copy()
df_experiment.Label.replace("Web.*", "Web Attack", regex=True, inplace=True)
df_experiment.Label.replace(r'.*Patator$', "Brute Force", regex=True,inplace=True)
df_experiment.Label.replace(["DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris"], "DoS", inplace=True)

class_attack = ['PortScan', 'Web Attack', 'Brute Force', 'DDoS', 'Bot','Infiltration', 'DoS', 'Heartbleed']
df_experiment.Label.replace(class_attack, value='attack', inplace=True)
df_experiment.Label.value_counts()

BENIGN    2271320
attack     556556
Name: Label, dtype: int64

In [8]:
# data split and preprocess
from sklearn.model_selection import train_test_split
import joblib

std_scaler,mm_scaler,le = joblib.load("model/std_mm_le_new.save")

y = df_experiment.Label
X = df_experiment.drop(columns='Label')
labels = y.unique()

X = std_scaler.transform(X)

classes = y.nunique()
print(X.shape)
print("number of labels:", classes)
print("instances per label\n", y.value_counts())
print("labels:", labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

print("after spliting the data:\n")
print("training data length:", len(X_train))
print("test data length:", len(X_test))

X_train = mm_scaler.transform(X_train)
X_test = mm_scaler.transform(X_test)

print("after MinMaxScaler")
print(X_train.shape)

#標籤編碼 可不跑
# y_train = le.transform(y_train)
# y_test = le.transform(y_test)

(2827876, 77)
number of labels: 2
instances per label
 BENIGN    2271320
attack     556556
Name: Label, dtype: int64
labels: ['BENIGN' 'attack']
after spliting the data:

training data length: 2120907
test data length: 706969
after MinMaxScaler
(2120907, 77)


# Random Forest

In [9]:
###### Random Forest
import joblib

rng = np.random.RandomState(42)

model = joblib.load("model/pkl/new_randomForest_32.pkl")
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(y_train_pred)
print(y_train_pred.shape)
print(y_test_pred)
print(y_test_pred.shape)

print("percentage of Anomaly:", (list(y_test_pred).count(1)/y_test_pred.shape[0])*100)
print("percentage of Legit:",(list(y_test_pred).count(0)/y_test_pred.shape[0])*100)

[0 0 0 ... 1 0 1]
(2120907,)
[0 0 1 ... 0 0 0]
(706969,)
percentage of Anomaly: 19.7059559895837
percentage of Legit: 80.2940440104163


In [14]:
y_train

2002012    BENIGN
1915100    BENIGN
1293022    BENIGN
1300105    BENIGN
2639033    BENIGN
            ...  
2479407    BENIGN
1365503    BENIGN
413800     attack
1147176    BENIGN
2226647    attack
Name: Label, Length: 2120907, dtype: object

In [17]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_train, y_train_pred)
tn, fp, fn, tp = cf_matrix.ravel()
cf_matrix

array([[1702181,    1309],
       [    596,  416821]], dtype=int64)